In [ ]:
!pip install torch_geometric

# Optional dependencies:
!pip install pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-2.1.0+cu121.html

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 33.2 MB/s eta 0:00:00
Looking in links: https://data.pyg.org/whl/torch-2.1.0+cu121.html
ERROR: Could not find a version that satisfies the requirement pyg_lib (from versions: none)
ERROR: No matching distribution found for pyg_lib


In [ ]:
!pip install git+https://github.com/leonardodalinky/pytorch_fpsample
!pip install .

  Cloning https://github.com/leonardodalinky/pytorch_fpsample to /tmp/pip-req-build-a_ebqkww
  Running command git clone --filter=blob:none --quiet https://github.com/leonardodalinky/pytorch_fpsample /tmp/pip-req-build-a_ebqkww
  Resolved https://github.com/leonardodalinky/pytorch_fpsample to commit 7bb2fda82d4f726c5c8dcaaa5bcd8f6d2546cea9
  Preparing metadata (setup.py) ... done
  Created wheel for torch_fpsample: filename=torch_fpsample-0.1.0-cp312-cp312-linux_x86_64.whl size=11623242 sha256=5174c60d2edb08dd6402c40652c73b6fe8a04305b6cdcffa5f5870da3c76d224
  Stored in directory: /tmp/pip-ephem-wheel-cache-49mshdz0/wheels/ec/d0/23/0f053a8d3d881003b5f0f4e80f0e7defa0f545b4ecd5516a38
Successfully built torch_fpsample
ERROR: Directory '.' is not installable. Neither 'setup.py' nor 'pyproject.toml' found.


In [ ]:
!pip install proteinshake

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.1/270.1 kB 7.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.9/69.9 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.3/68.3 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 47.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.8/15.8 MB 48.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.1/36.1 MB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.3/175.3 kB 18.0 MB/s eta 0:00:00
  Created wheel for freesasa: filename=freesasa-2.2.1-cp312-cp312-linux_x86_64.whl size=951900 sha256=1a0480e6a88bd6c40ebefe8131861636e596b6ac61dc5e3e8ff88d71ab741515
  Stored in directory: /root/.cache/pip/wheels/d5/ba/ad/c7be874883ac321912d0442a2e8ff4208d0c3bf3bf94952387
Successfully built freesasa


In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from proteinshake.tasks import EnzymeClassTask
import torch_fpsample
import h5py
import tqdm

task = EnzymeClassTask().to_point().torch()

100%|██████████| 9.00k/9.00k [00:05<00:00, 18.3MiB/s]


Unzipping...


Converting: 100%|██████████| 15603/15603 [00:19<00:00, 803.23it/s]


In [ ]:
class PointCloudTransformFPS:
    def __init__(self, max_points=1024):
        self.max_points = max_points

    def __call__(self, data_tuple):
        numerical_data, metadata_dict = data_tuple
        coords = numerical_data[:, :3].detach().clone().float()
        labels = numerical_data[:, 3].detach().clone().long()

        num_points_original = coords.shape[0]

        if num_points_original > self.max_points:
            # Farthest Point Sampling
            _, indices = torch_fpsample.sample(coords.unsqueeze(0), self.max_points)
            indices = indices.squeeze(0).long()
            sampled_coords = coords[indices]
            sampled_labels = labels[indices]
        else:
            padding_needed = self.max_points - num_points_original
            sampled_coords = F.pad(coords, (0, 0, 0, padding_needed), "constant", 0)
            sampled_labels = F.pad(labels, (0, padding_needed), "constant", 20)

        # Create mask for padding points
        num_real_points = min(num_points_original, self.max_points)
        mask = torch.zeros(self.max_points, dtype=torch.bool)
        mask[:num_real_points] = True


        ec_string = metadata_dict['protein']['EC']
        ec_class = int(ec_string[0]) if ec_string[0].isdigit() else 0
        ec_class = ec_class - 1

        return Data(
            pos=sampled_coords,
            x=sampled_labels,
            mask=mask,
            y=torch.tensor([ec_class], dtype=torch.long) # Nhãn EC đã xử lý
        )

In [ ]:
class ProteinDataset(torch.utils.data.Dataset):
    def __init__(self, original_dataset, transform):
        self.original_dataset = original_dataset
        self.transform = transform
    def __len__(self):
        return len(self.original_dataset)
    def __getitem__(self, idx):
        return self.transform(self.original_dataset[idx])

In [ ]:
transform = PointCloudTransformFPS(max_points=512)
train_dataset = ProteinDataset(task.train, transform)
val_dataset = ProteinDataset(task.val, transform)
test_dataset = ProteinDataset(task.test, transform)

In [ ]:
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)
train_dataset[0]

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Data(x=[512], y=[1], pos=[512, 3], mask=[512])

In [ ]:
import h5py
import numpy as np

def save_to_h5(dataset, filename):
    with h5py.File(filename, "w") as f:
        # Create datasets with variable size
        coords_dset = f.create_dataset("coords", (len(dataset), 512, 3), dtype="float32")
        labels_dset = f.create_dataset("labels", (len(dataset), 512), dtype="int64")
        mask_dset   = f.create_dataset("mask",   (len(dataset), 512), dtype="bool")
        targets_dset = f.create_dataset("targets", (len(dataset),), dtype="int64")

        for i, data in enumerate(dataset):
            coords_dset[i] = data.pos.numpy()
            labels_dset[i] = data.x.numpy()
            mask_dset[i]   = data.mask.numpy()
            targets_dset[i] = data.y.item()

    print(f"Saved {len(dataset)} samples to {filename}")

save_to_h5(train_dataset, "protein_train.h5")
save_to_h5(val_dataset, "protein_val.h5")
save_to_h5(test_dataset, "protein_test.h5")

Saved 14042 samples to protein_train.h5
Saved 780 samples to protein_val.h5
Saved 781 samples to protein_test.h5
